In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('df_rios_precipitaciones_limpio.csv')
df

,Unnamed: 0,date,rio_id,lat,lon,altura_value,precipitaciones_value,es_null
0,0,2020-10-05,8-8,-25.583333,-53.983333,0.30,0.0,False
1,1,2020-10-06,8-8,-25.583333,-53.983333,0.67,0.0,False
2,2,2020-10-07,8-8,-25.583333,-53.983333,1.20,0.0,False
3,3,2020-10-08,8-8,-25.583333,-53.983333,1.08,3.0,False
4,4,2020-10-09,8-8,-25.583333,-53.983333,0.96,8.1,False
...,...,...,...,...,...,...,...,...
358121,358121,2025-10-01,37597-6622,-34.216389,-64.386111,1.11,NaN,True
358122,358122,2025-10-02,37597-6622,-34.216389,-64.386111,1.12,NaN,True
358123,358123,2025-10-03,37597-6622,-34.216389,-64.386111,1.11,NaN,True
358124,358124,2025-10-04,37597-6622,-34.216389,-64.386111,1.11,NaN,True


In [23]:
df = df.drop(columns=["Unnamed: 0"])

In [24]:
#Tirando abajo todas las fechas antesesoras del 1/1/2023. Ya que los datos de atras son muy pobres.
df["date"] = pd.to_datetime(df["date"])
df = df[df['date'] >= '2023-01-01']

In [25]:
print(f"La cantidad de rios: {len(df["rio_id"].unique())}")

print(f"La cantidad de filas con valores de prec. nulos: \n {df["es_null"].value_counts()}")
df

La cantidad de rios: 396
La cantidad de filas con valores de prec. nulos: 
 es_null
False    257295
True       6986
Name: count, dtype: int64


,date,rio_id,lat,lon,altura_value,precipitaciones_value,es_null
814,2023-01-01,8-8,-25.583333,-53.983333,-0.06,0.2,False
815,2023-01-02,8-8,-25.583333,-53.983333,0.01,46.5,False
816,2023-01-03,8-8,-25.583333,-53.983333,-0.02,20.8,False
817,2023-01-04,8-8,-25.583333,-53.983333,0.64,5.5,False
818,2023-01-05,8-8,-25.583333,-53.983333,0.56,0.0,False
...,...,...,...,...,...,...,...
358121,2025-10-01,37597-6622,-34.216389,-64.386111,1.11,NaN,True
358122,2025-10-02,37597-6622,-34.216389,-64.386111,1.12,NaN,True
358123,2025-10-03,37597-6622,-34.216389,-64.386111,1.11,NaN,True
358124,2025-10-04,37597-6622,-34.216389,-64.386111,1.11,NaN,True


In [34]:
# 1. Identificar los rio_id que tienen al menos un valor "es_null" en True
# Se puede usar unique() en la serie filtrada para obtener la lista de IDs.
# El método .any() dentro de un .groupby() también sería una opción.
rios_a_eliminar = df[df['es_null'] == True]['rio_id'].unique()

# 2. Filtrar el DataFrame original para mantener solo los ríos que NO están en la lista de ríos a eliminar
# Se usa el operador `~` para negar la condición, junto con .isin()
df = df[~df['rio_id'].isin(rios_a_eliminar)]

print(f"La nueva cantidad de rios: {len(df['rio_id'].unique())}")
print(f"La nueva cantidad de filas con valores de prec. nulos: \n {df['es_null'].value_counts()}")


La nueva cantidad de rios: 379
La nueva cantidad de filas con valores de prec. nulos: 
 es_null
False    257295
Name: count, dtype: int64


In [ ]:
df = df.drop(columns= "es_null")


In [50]:
print(f"La cantidad de rios es de: 379")
print(f"La cantidad de dias registrados 'por rio' es: {len(df.date.unique())}")
print(f"La cantidad de filas es de: {len(df)}")
print(f"Deberían ser 382.411...")

La cantidad de rios es de: 379
La cantidad de dias registrados 'por rio' es: 1009
La cantidad de filas es de: 257295
Deberían ser 382.411...


### Este codigo, nos dice cuantas filas se comió la API del gobierno a la hora de hacer los registros por cada rio.

In [54]:
# 1. Asegurar que la columna 'date' sea el índice y de tipo Datetime
df['date'] = pd.to_datetime(df['date'])

# 2. Definir el rango de fechas completo y continuo
fecha_minima = df['date'].min()
fecha_maxima = df['date'].max()
rango_completo = pd.date_range(start=fecha_minima, end=fecha_maxima, freq='D')

# 3. Función de Detección y Conteo de Huecos
def contar_dias_faltantes(grupo_df):
    """Devuelve el número de días faltantes para un río."""
    
    # Establecer la fecha como índice para este río
    df_temporal = grupo_df.set_index('date')
    
    # Aplicar reindexación: rellena las fechas faltantes con NaN
    df_reindexado = df_temporal.reindex(rango_completo)
    
    # Contar los lugares donde 'altura_value' es nulo
    dias_faltantes = df_reindexado['altura_value'].isnull().sum()
    
    return dias_faltantes


# 4. Aplicar la función a cada río y obtener el conteo
print("Iniciando el conteo de días faltantes...")
conteo_huecos = df.groupby('rio_id').apply(contar_dias_faltantes)


# 5. Crear y Mostrar la Tabla de Resultados
df_conteo_final = conteo_huecos.reset_index(name='dias_faltantes')

# Filtrar: Mostrar solo los ríos que tienen huecos (> 0 días faltantes)
df_conteo_final = df_conteo_final[df_conteo_final['dias_faltantes'] > 0]

# Ordenar de mayor a menor falta de datos
df_conteo_final = df_conteo_final.sort_values(by='dias_faltantes', ascending=False)

print("\n--- RÍOS CON DÍAS DE REGISTRO FALTANTES ---")
print(f"Total de ríos analizados: {df['rio_id'].nunique()}")
print(f"Total de ríos con huecos: {len(df_conteo_final)}")
print("\nTabla de Registros Faltantes:")
df_conteo_final.to_csv('df_conteo_faltantes.csv')



Iniciando el conteo de días faltantes...

--- RÍOS CON DÍAS DE REGISTRO FALTANTES ---
Total de ríos analizados: 379
Total de ríos con huecos: 372

Tabla de Registros Faltantes:


C:\Users\COMPU\AppData\Local\Temp\ipykernel_8056\2139946606.py:27: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  conteo_huecos = df.groupby('rio_id').apply(contar_dias_faltantes)


In [59]:
import pandas as pd

# Asumiendo que tu dataframe se llama df
df = pd.read_csv('df_conteo_faltantes.csv')

# Definir los rangos de 100 en 100
rangos = []
rangos.append((0, 100, "Menor a 100"))

# Rangos de 100 en 100
for i in range(100, 900, 100):
    rangos.append((i, i+100, f"{i}-{i+100}"))

rangos.append((900, float('inf'), "Mayor a 900"))

# Contar registros por rango
print("="*60)
print("DISTRIBUCIÓN DE REGISTROS POR RANGO DE DÍAS FALTANTES")
print("="*60)

for inicio, fin, etiqueta in rangos:
    if fin == float('inf'):
        cantidad = len(df[df['dias_faltantes'] > inicio])
    else:
        cantidad = len(df[(df['dias_faltantes'] >= inicio) & (df['dias_faltantes'] < fin)])
    
    porcentaje = (cantidad / len(df)) * 100
    print(f"{etiqueta:>15}: {cantidad:>8,} registros ({porcentaje:>6.2f}%)")

print("="*60)
print(f"{'TOTAL':>15}: {len(df):>8,} registros")
print("="*60)

DISTRIBUCIÓN DE REGISTROS POR RANGO DE DÍAS FALTANTES
    Menor a 100:      134 registros ( 36.02%)
        100-200:       55 registros ( 14.78%)
        200-300:       16 registros (  4.30%)
        300-400:       11 registros (  2.96%)
        400-500:        3 registros (  0.81%)
        500-600:        8 registros (  2.15%)
        600-700:      110 registros ( 29.57%)
        700-800:       15 registros (  4.03%)
        800-900:        7 registros (  1.88%)
    Mayor a 900:       13 registros (  3.49%)
          TOTAL:      372 registros
